In [1]:
import requests

In [2]:
review_url = 'https://www.airlinequality.com/airline-reviews/british-airways/'

In [3]:
from bs4 import BeautifulSoup

In [4]:
total_pages = 301

In [5]:
rating_dates = []
rating_titles = []
rating_ids = []
rating_values = []

for page in range (1,total_pages):
    page_url = review_url + f"page/{page}/"
    page_response = requests.get(page_url)
    page_soup = BeautifulSoup(page_response.content, "html.parser")
    rating_date = page_soup.find_all('time', {'itemprop':'datePublished'})
    rating_title = page_soup.find_all('h2', {'class':'text_header'})
    rating_id = page_soup.find_all('span', {'itemprop':'name'})
    the_rating_value = page_soup.find_all('span', {'itemprop':'ratingValue'})
    rating_value = the_rating_value[1:]
    rating_values.append(rating_value)
    rating_dates.append(rating_date)
    rating_titles.append(rating_title)
    rating_ids.append(rating_id)
    

rating_date_tag = sum(rating_dates,[])
rating_title_tag = sum(rating_titles,[])
rating_id_tag = sum(rating_ids,[])
rating_value_tag = sum(rating_values,[])

In [6]:
len(rating_value_tag)

3000

In [7]:
rate_date = []
rate_title = []
rate_id = []
rate_value = []
for b,c,d,a in zip (rating_date_tag, rating_title_tag, rating_id_tag, rating_value_tag):
    rate_date.append(b.text.strip())
    rate_title.append(c.text.strip())
    rate_id.append(d.text.strip())
    rate_value.append(a.text.strip())

In [8]:
len(rate_value)

3000

In [9]:
page_urls = []
for page in range (1,total_pages):
    page_ur = review_url + f"page/{page}/"
    page_urls.append(page_ur)

In [10]:
review_values = {}

for page_url in page_urls:
    page_response = requests.get(page_url)
    page_soup = BeautifulSoup(page_response.content, "html.parser")
    tables = page_soup.find_all('table')

    for i, table in enumerate(tables[1:], 1):
        rows = table.find_all('tr')

        # Reset the all_keys set for each table
        all_keys = set(review_values.keys())

        for row in rows:
            cells = row.find_all('td')

            if len(cells) == 2:
                key = cells[0].get_text(strip=True)
                value = cells[1].get_text(strip=True)

                if 'review-rating-stars' in cells[1].get('class', []):
                    span_elements = cells[1].find_all('span', class_='star fill')
                    if span_elements:
                        max_value = max(int(span.get_text(strip=True)) for span in span_elements)
                        value = str(max_value)

                if key not in review_values:
                    review_values[key] = []

                review_values[key].append(value)

                # Remove key from all_keys set if it appears in the current table
                all_keys.discard(key)

        # Check if any expected keys are missing in the current table and assign '0' as the value
        for key in all_keys:
            review_values[key].append('0')

In [11]:
len(rating_value_tag)

3000

In [12]:
len(rating_value_tag)

3000

In [13]:
len(review_values['Inflight Entertainment'])

3000

In [14]:
all_keys

{'Aircraft',
 'Date Flown',
 'Ground Service',
 'Route',
 'Type Of Traveller',
 'Wifi & Connectivity'}

In [15]:
len(review_values['Seat Comfort'])

3000

In [16]:
traveller_type = review_values['Type Of Traveller']
seat_type = review_values['Seat Type']
route = review_values['Route']
date_flown = review_values['Date Flown']
seat_comfort = review_values['Seat Comfort']
flight_entertainment = review_values['Inflight Entertainment']
cabin_staff_service = review_values['Cabin Staff Service']
food_beverage = review_values['Food & Beverages']
ground_service = review_values['Ground Service']
connectivity = review_values['Wifi & Connectivity']
value_for_money = review_values['Value For Money']
recommended = review_values['Recommended']

In [17]:
len(flight_entertainment)

3000

In [18]:
import pandas as pd

In [19]:
opinions = pd.DataFrame(
            {'rate_id': rate_id,
             'rate_date': rate_date,
             'rate_title': rate_title,
             'rate_value': rate_value,
             'traveller_type': traveller_type,
             'seat_type': seat_type,
             'route': route,
             'date_flown': date_flown,
             'seat_comfort': seat_comfort,
             'cabin_staff_service': cabin_staff_service,
             'food_beverage': food_beverage,
             'ground_service': ground_service,
             'value_for_money': value_for_money,
             'recommended': recommended,  
            })

In [20]:
opinions

,rate_id,rate_date,rate_title,rate_value,traveller_type,seat_type,route,date_flown,seat_comfort,cabin_staff_service,food_beverage,ground_service,value_for_money,recommended
0,Bervin Hedman,6th July 2023,"""short-changing passengers""",4,Family Leisure,Premium Economy,Kingston to London,June 2023,5,4,4,3,3,no
1,Alastair Cockburn,5th July 2023,"""Economy is absolutely awful""",1,Couple Leisure,Economy Class,London Heathrow to Cape Town,June 2023,1,3,1,3,2,no
2,S Carlsen,5th July 2023,"""Shocking customer service""",1,Couple Leisure,Economy Class,London Heathrow to Ibiza,July 2023,0,0,0,1,1,no
3,A Diamantopoulos,4th July 2023,"""no representative to help""",3,Business,Business Class,Washington to London,June 2023,3,3,3,1,1,no
4,Carlos Whilhelm,3rd July 2023,"""Boarding was chaotic""",3,Couple Leisure,Economy Class,London Heathrow to Naples,June 2023,1,4,1,2,2,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,C Cutts,9th March 2015,British Airways customer review,9,0,Business Class,0,0,4,5,5,0,4,yes
2996,R Hanstock,9th March 2015,British Airways customer review,4,0,Economy Class,0,0,3,4,2,0,4,no
2997,Desmond Whelan,9th March 2015,British Airways customer review,2,0,Business Class,0,0,1,2,1,0,2,no
2998,Richard Lightbody,9th March 2015,British Airways customer review,8,0,Business Class,0,0,3,5,5,0,5,yes


In [21]:
opinions.to_csv('airways.csv', index=None)